In [2]:
import pandas as pd

In [3]:
inputFile = "OPP_proteins.tsv"

In [4]:
df_OPPprot = pd.read_csv(inputFile, delimiter="\t")

In [5]:
df_OPPprot["Stn_depth"] = df_OPPprot["station_id"].astype(str) + "_" + df_OPPprot["depth_m"].astype(str)

In [6]:
df_OPPprot.columns

Index(['sample_id', 'cruise_id', 'station_id', 'latitude_dd', 'longitude_dd',
       'depth_m', 'date_y-m-d', 'time_h-m-s', 'maximum_filter_size_microns',
       'minimum_filter_size_microns', 'protein_id', 'protein_name',
       'spectral_count', 'molecular_weight_kDa', 'ncbi_id', 'ncbi_name',
       'kegg_id', 'kegg_description', 'kegg_pathway', 'pfams_id', 'pfams_name',
       'uniprot_id', 'enzyme_comm_id', 'other_identified_proteins',
       'Stn_depth'],
      dtype='object')

In [7]:
sampleInfo = df_OPPprot[['sample_id', 'cruise_id', 'station_id', 'latitude_dd', 'longitude_dd',
       'depth_m', 'date_y-m-d', 'time_h-m-s', 'maximum_filter_size_microns',
       'minimum_filter_size_microns','Stn_depth']]

In [8]:
len(sampleInfo)

53506

In [9]:
UsampleInfo = sampleInfo.drop_duplicates()

In [10]:
len(UsampleInfo)

40

In [11]:
proteinInfo = df_OPPprot[['protein_id', 'protein_name',
        'molecular_weight_kDa', 'ncbi_id', 'ncbi_name',
       'kegg_id', 'kegg_description', 'kegg_pathway', 'pfams_id', 'pfams_name',
       'uniprot_id', 'enzyme_comm_id', 'other_identified_proteins']]

In [12]:
len(proteinInfo)

53506

In [13]:
UproteinInfo = proteinInfo.drop_duplicates()

In [14]:
len(UproteinInfo)

7999

In [15]:
subdf = df_OPPprot[['protein_id', 'Stn_depth', 'spectral_count']]

In [16]:
subdf_pivot = subdf.pivot(index='protein_id', columns='Stn_depth', values='spectral_count')

In [17]:
subdf_pivot = subdf_pivot.fillna(0)

In [18]:
subdf_pivot = subdf_pivot.reset_index()

In [19]:
subdf_pivot.columns

Index(['protein_id', '12_120', '12_1900', '12_300', '12_40', '1_120', '1_200',
       '1_300', '1_400', '1_50', '1_90', '3_120', '3_150', '3_200', '3_250',
       '3_300', '3_40', '3_400', '3_500', '3_550', '3_60', '3_600', '3_800',
       '5_120', '5_20', '5_200', '5_300', '5_400', '5_50', '5_500', '5_600',
       '5_80', '6_200', '6_40', '6_80', '8_200', '8_40', '8_70', '9_380',
       '9_40', '9_70'],
      dtype='object', name='Stn_depth')

In [24]:
colList = subdf_pivot.columns.tolist()
colList = colList[1:]

In [22]:
subdf_melt = pd.melt(subdf_pivot, id_vars=["protein_id"], value_vars=colList)

In [46]:
subdf_melt = subdf_melt.rename(columns={"value":"spectral_count"})

In [48]:
len(subdf_melt)

319960

In [63]:
df_mergeProtein = pd.merge(subdf_melt, UproteinInfo, left_on="protein_id", right_on="protein_id")

In [64]:
len(df_mergeProtein)

319960

In [69]:
df_mergeProteinSample = pd.merge(df_mergeProtein, UsampleInfo, left_on='Stn_depth', right_on='Stn_depth')

In [70]:
len(df_mergeProteinSample)

319960

In [72]:
df_out = df_mergeProteinSample[['sample_id', 'cruise_id', 'station_id', 'latitude_dd', 'longitude_dd',
       'depth_m', 'date_y-m-d', 'time_h-m-s', 'maximum_filter_size_microns',
       'minimum_filter_size_microns', 'protein_id', 'protein_name',
       'spectral_count', 'molecular_weight_kDa', 'ncbi_id', 'ncbi_name',
       'kegg_id', 'kegg_description', 'kegg_pathway', 'pfams_id', 'pfams_name',
       'uniprot_id', 'enzyme_comm_id', 'other_identified_proteins']]

In [73]:
len(df_out)

319960

In [75]:
df_out.to_csv("OPP_proteins_update.tsv", index=False, sep="\t")